In [1]:
import numpy as np
import pickle
import torch 
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from fastprogress import progress_bar

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
x = np.load('data/board_states.pkl')
y = np.load('data/results.pkl')

In [4]:
class ChessDataset(Dataset):

    def __init__(self, x, y):  
        self.x = x.astype('float')
        self.y = y.astype('long')

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
    
dataset = ChessDataset(x, y)

In [5]:
dataloader = DataLoader(
    dataset, 
    batch_size=128,
    shuffle=True, 
)

In [6]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(12, 8, 3)
#         self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 6, 3)
        self.fc1 = nn.Linear(6*4*4, 3)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(-1, 6*4*4)
        x = self.fc1(x)
        return x
    
model = Model()
model.to(device)

Model(
  (conv1): Conv2d(12, 8, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(8, 6, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=96, out_features=3, bias=True)
)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

In [10]:
for epoch in range(10):
    running_loss = 0.0
    for i, (inputs, results) in enumerate(dataloader):
        model.train()
        
        inputs, results = inputs.type(torch.FloatTensor), results.type(torch.LongTensor)
        inputs, results = inputs.to(device), results.to(device)
                
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, results)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch:{epoch+1} Loss:{running_loss/len(x)}")

Epoch:1 Loss:0.006738377013450939
Epoch:2 Loss:0.006719848508648516
Epoch:3 Loss:0.006704345689573102
Epoch:4 Loss:0.006693004320240592
Epoch:5 Loss:0.006681616973452857
Epoch:6 Loss:0.006671758235413455
Epoch:7 Loss:0.006661331880021247
Epoch:8 Loss:0.006651883303207835
Epoch:9 Loss:0.006644869781348503
Epoch:10 Loss:0.006633986731734066


In [ ]:
# TODO 
# add more data
# Do inference as an agent